# Pemodelan

In [42]:
#import lib
import csv
import pandas as pd
import numpy as np
import string
import re
import nltk

#
import matplotlib.pyplot as plt
import seaborn as sns

# Analisi Sentimen

### Source CODE dari
https://github.com/commitunuja/analisis-sentimen-naive-bayes-tf-idf/blob/master/.ipynb_checkpoints/Untitled-checkpoint.ipynb

In [43]:
# %run S_4_Analisis_sentimen.ipynb

# Naive Bayes Classifier in Python KAGGLE
https://www.kaggle.com/prashant111/naive-bayes-classifier-in-python

In [44]:
datax = pd.read_csv('E:\Program\[2] Program\AnSent\Data\Data_5_Lex_sen\Lex_Indihome.csv')

# menghapus kolom
del datax['Unnamed: 0.1']
del datax['Unnamed: 0']
# datas = datax.drop(['Unnamed: 0.1', 'Unnamed: 0', 'Data Teks', 'Data Teks_CaseFolding', 'Data Teks_clean_alay', 'Data Teks_Token', 'Data Teks_Stopword', 'Jumlah_polarity'])
# data.rename( columns={'Unnamed: 0':'Indeks'}, inplace=True )
data = datax.copy()


In [45]:
# data = data.astype({'Sentimen' : 'category'})
# data = data.astype({'Data Teks_Stemming' : 'string'})
# data.dtypes

### Prediction TF-IDF

In [46]:
# convert list formated string to list
import ast
import numpy as np
index = 0

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

data['Data Teks_List'] = data['Data Teks_Token'].apply(convert_text_list)


print(data['Data Teks_List'][index])

print('\ntype : ', type(data['Data Teks_List']))

['sudah', 'dilakukan', 'sesuai', 'pesan', 'belum', 'bisa']

type :  <class 'pandas.core.series.Series'>


In [47]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

data["TF_dict"] = data['Data Teks_List'].apply(calc_TF)

data["TF_dict"].head()

0    {'sudah': 0.16666666666666666, 'dilakukan': 0....
1                                           {'p': 1.0}
2    {'kak': 0.041666666666666664, 'dila': 0.041666...
3    {'ini': 0.14285714285714285, 'kapan': 0.071428...
4    {'oy': 0.08333333333333333, 'mola': 0.08333333...
Name: TF_dict, dtype: object

In [48]:
print('%20s' % "term", "\t", "TF\n")
for key in data["TF_dict"][index]:
    print('%20s' % key, "\t", data["TF_dict"][index][key])

                term 	 TF

               sudah 	 0.16666666666666666
           dilakukan 	 0.16666666666666666
              sesuai 	 0.16666666666666666
               pesan 	 0.16666666666666666
               belum 	 0.16666666666666666
                bisa 	 0.16666666666666666


In [49]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(data["TF_dict"])

In [50]:
n_document = len(data)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [51]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
data["TF-IDF_dict"] = data["TF_dict"].apply(calc_TF_IDF)

In [52]:
print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in data["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", data["TF_dict"][index][key] ,"\t" , data["TF-IDF_dict"][index][key])

                term 	         TF 	              TF-IDF

               sudah 	 0.16666666666666666 	 0.2542200919300053
           dilakukan 	 0.16666666666666666 	 0.6251534177936807
              sesuai 	 0.16666666666666666 	 0.8623381401674486
               pesan 	 0.16666666666666666 	 0.2018062581670243
               belum 	 0.16666666666666666 	 0.5516581183661684
                bisa 	 0.16666666666666666 	 0.31884197335297587


In [53]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

data["TF_IDF_Vec"] = data["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(data["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(data["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[0.0, 0.0, 0.2018062581670243, 0.0, 0.0, 0.0, 0.2542200919300053, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.31884197335297587, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


### Save TF-IDF

In [54]:
data.to_csv('E:\Program\[2] Program\AnSent\Data\Data_8_Pemod\Mod_Indihome.csv')
# data

In [55]:
data['sentimen'].value_counts()

negatif    3880
positif    2648
netral      537
Name: sentimen, dtype: int64

# NBC
https://github.com/rizkiamanullah/Analisis-Sentiment-Naive-Bayes/blob/main/rizkiamanullah-Analysis-Sentiment-NaiveB.ipynb

# ini penting (confusion matrix)
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

In [56]:
# copy dataframe
data_s = datax.copy()
# data_s.head(5)

In [57]:
# menghapus dataframe netral
data_s = data_s[data_s.sentimen != 'netral']
# data_s.head(5)

In [58]:
# Conversi nilai
def convernilai(data):
    if data == 'positif':
        return 1
    elif data == 'negatif':
        return -1

data_s['label'] = data_s['sentimen'].apply(lambda x: convernilai(x))
# data_s

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

vectorizer = TfidfVectorizer()
v_data= vectorizer.fit_transform(data_s['Data Teks_clean_alay'].values.astype('U'))
print(v_data)
v_data= vectorizer.fit_transform(data_s['Data Teks_clean_alay'].values.astype('U')).toarray()

  (0, 1107)	0.2900543311693539
  (0, 856)	0.4342205774997905
  (0, 6282)	0.21951162489795123
  (0, 7267)	0.6219845912013412
  (0, 1938)	0.4777814385244335
  (0, 7599)	0.2529736881375842
  (1, 8704)	0.07233936411316859
  (1, 2256)	0.17331042627839335
  (1, 7219)	0.19222131111819674
  (1, 7409)	0.19022350604834604
  (1, 1604)	0.12006896804403233
  (1, 1997)	0.2992334941998413
  (1, 6192)	0.16610109104908377
  (1, 3302)	0.09380290684818213
  (1, 6900)	0.12283719506684926
  (1, 7279)	0.2030005397797552
  (1, 8545)	0.17577236509589403
  (1, 3477)	0.16310333297035656
  (1, 7129)	0.18698397515946064
  (1, 2950)	0.1388929994226607
  (1, 2393)	0.27004197817123304
  (1, 5921)	0.1422204488599797
  (1, 3902)	0.10717172470414077
  (1, 1891)	0.22228069379302487
  (1, 8402)	0.2992334941998413
  :	:
  (6526, 3287)	0.17112902908580527
  (6526, 7841)	0.36196408559362897
  (6526, 7239)	0.19212764998055495
  (6526, 3885)	0.17949269241597407
  (6526, 7193)	0.15899876421771159
  (6526, 573)	0.15940170606390

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(v_data, data_s['label'], test_size=0.2, random_state=0)
clf = MultinomialNB()
clf.fit(X_train,y_train)
y_preds = clf.predict(X_test)

print('Banyak data train :',len(X_train))
print('Banyak data test  :',len(X_test))

Banyak data train : 5222
Banyak data test  : 1306


In [61]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_preds, labels=[1,-1]))
print(classification_report(y_test,y_preds))
print('nilai akurasinya adalah {:.2f}'.format((accuracy_score(y_test,y_preds)*100)))

[[357 170]
 [110 669]]
              precision    recall  f1-score   support

          -1       0.80      0.86      0.83       779
           1       0.76      0.68      0.72       527

    accuracy                           0.79      1306
   macro avg       0.78      0.77      0.77      1306
weighted avg       0.78      0.79      0.78      1306

nilai akurasinya adalah 78.56


# Pemodelan

In [62]:
import pandas as pd 
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,classification_report

In [63]:
#Data
data = pd.read_csv('E:\Program\[2] Program\AnSent\Data\Data_5_Lex_sen\Lex_Firstmediacares.csv', sep=(','))
data.head(5)

,Unnamed: 0,Unnamed: 0.1,Tanggal,User ID,Data Teks,Data Teks_CaseFolding,Data Teks_clean_alay,Data Teks_Token,Data Teks_Stopword,Data Teks_Stemming,jumlah_polarity,sentimen
0,25012,25012,2021-10-19 14:48:25+00:00,FirstMediaCares,"@adoths Selamat malam FIRST People, Mohon maa...",selamat malam first people mohon maaf atas ket...,selamat malam first people mohon maaf atas ket...,"['selamat', 'malam', 'first', 'people', 'mohon...","['selamat', 'malam', 'first', 'people', 'mohon...",selamat malam first people mohon maaf ketidakn...,-10,negatif
1,1784,1784,2021-10-30 16:40:56+00:00,FirstMediaCares,@DanielVHD Mohon check DM anda.,mohon check dm anda,mohon memeriksa pesan kamu,"['mohon', 'memeriksa', 'pesan', 'kamu']","['mohon', 'memeriksa', 'pesan']",mohon memeriksa pesan,-6,negatif
2,37330,37330,2021-10-13 05:30:28+00:00,FirstMediaCares,@akhyaroy Hi First People. Kami sudah merespon...,hi first people kami sudah merespon via dm sil...,hi first people kami sudah merespon via pesan...,"['hi', 'first', 'people', 'kami', 'sudah', 'me...","['hi', 'first', 'people', 'merespon', 'via', '...",hi first people merespon via pesan silakan dic...,8,positif
3,3000,3000,2021-10-30 05:43:35+00:00,FirstMediaCares,@soeguss Adapun estimasi perbaikan hingga 30 O...,adapun estimasi perbaikan hingga oktober puk...,adapun estimasi perbaikan hingga oktober puk...,"['adapun', 'estimasi', 'perbaikan', 'hingga', ...","['estimasi', 'perbaikan', 'oktober', 'wib', 'i...",estimasi perbaikan oktober wib informasi probl...,-5,negatif
4,8225,8225,2021-10-28 08:20:58+00:00,DediBudiawan2,@FirstMediaCares Baik tks,baik tks,baik tks,"['baik', 'tks']",['tks'],tks,2,positif


In [64]:
# menghapus dataframe netral
data = data[data_s.sentimen != 'netral']

C:\Users\ASUS\AppData\Local\Temp/ipykernel_5132/2581358189.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data[data_s.sentimen != 'netral']


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
A=data['Data Teks_Stemming']
b=data['sentimen']

a_train, a_test, b_train, b_test = train_test_split(A,b,test_size = 0.20, random_state = 42)

print('Banyak data train :',len(a_train))
print('Banyak data test  :',len(a_test))

# Naive Bayes model

In [ ]:
# modelnb = GaussianNB()

# #input data train pada fungsi klasifikasi NB
# nbtrain = modelnb.fit(a_train, b_train)

In [ ]:
# #Menentukan hasil prediksi dari a_test
# b_pred = nbtrain.predict(a_test, b_train)
# b_pred

In [ ]:
# np.array(b_test)

In [ ]:
# #Menentukan probabilitas hasil prediksi
# nbtrain.predict_proba(a_test)

In [ ]:
# #confusion matrix
# from sklearn.metrics import confusion

# confusion_matrix(b_test, b_pred)

In [ ]:
# #Menerapkan hasil confusion matrix
# y_actual = pd.Series

In [ ]:
# from sklearn.metrics import classification_report
# print(classification_report(b_test, b_pred))

# Lexicon + KNN

### Source CODE
https://github.com/commitunuja/analisis-sentimen-naive-bayes-tf-idf/blob/master/klasifikasi_akurasi.ipynb

In [ ]:
tvec_lxn=TfidfVectorizer()
clf1_lxn= KNeighborsClassifier()


#Pipeline
model_lxn = Pipeline([('vectorizer',tvec_lxn),
                 ('classifier',clf1_lxn)])


model_lxn.fit(a_test,b_test)

In [ ]:
hasil_lxn = model_lxn.predict(a_test)

In [ ]:
print(confusion_matrix(b_test, hasil_lxn))
print(classification_report(hasil_lxn, b_test))

In [ ]:
print("Accuracy Score: {:.2f}".format((accuracy_score(hasil_lxn,b_test)*100)))
print("Precision Score: {:.2f}".format(precision_score(hasil_lxn, b_test, average='macro')))
print("Recall Score: {:.2f}".format(recall_score(hasil_lxn, b_test, average='macro')))
